#  Assignement 6

In [1]:

#  read California housing data
# ! wget https://raw.githubusercontent.com/ageron/handson-ml2/master/datasets/housing/housing.csv
! python -m wget https://raw.githubusercontent.com/alexeygrigorev/datasets/master/housing.csv



Saved under housing.csv


In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
df_House=pd.read_csv('housing.csv')
df_House.head()

,longitude,latitude,housing_median_age,total_rooms,total_bedrooms,population,households,median_income,median_house_value,ocean_proximity
0,-122.23,37.88,41.0,880.0,129.0,322.0,126.0,8.3252,452600.0,NEAR BAY
1,-122.22,37.86,21.0,7099.0,1106.0,2401.0,1138.0,8.3014,358500.0,NEAR BAY
2,-122.24,37.85,52.0,1467.0,190.0,496.0,177.0,7.2574,352100.0,NEAR BAY
3,-122.25,37.85,52.0,1274.0,235.0,558.0,219.0,5.6431,341300.0,NEAR BAY
4,-122.25,37.85,52.0,1627.0,280.0,565.0,259.0,3.8462,342200.0,NEAR BAY


# Data Prepration
keep only the records where ocean_proximity is either '<1H OCEAN' or 'INLAND'

In [3]:
df_House=df_House[df_House['ocean_proximity'].isin(['INLAND','<1H OCEAN'])]

In [4]:
# keep only the records where ocean_proximity is either '<1H OCEAN' or 'INLAND'":
df_House[df_House['ocean_proximity'].isin(['<1H OCEAN','INLAND'])]


,longitude,latitude,housing_median_age,total_rooms,total_bedrooms,population,households,median_income,median_house_value,ocean_proximity
701,-121.97,37.64,32.0,1283.0,194.0,485.0,171.0,6.0574,431000.0,<1H OCEAN
830,-121.99,37.61,9.0,3666.0,711.0,2341.0,703.0,4.6458,217000.0,<1H OCEAN
859,-121.97,37.57,21.0,4342.0,783.0,2172.0,789.0,4.6146,247600.0,<1H OCEAN
860,-121.96,37.58,15.0,3575.0,597.0,1777.0,559.0,5.7192,283500.0,<1H OCEAN
861,-121.98,37.58,20.0,4126.0,1031.0,2079.0,975.0,3.6832,216900.0,<1H OCEAN
...,...,...,...,...,...,...,...,...,...,...
20635,-121.09,39.48,25.0,1665.0,374.0,845.0,330.0,1.5603,78100.0,INLAND
20636,-121.21,39.49,18.0,697.0,150.0,356.0,114.0,2.5568,77100.0,INLAND
20637,-121.22,39.43,17.0,2254.0,485.0,1007.0,433.0,1.7000,92300.0,INLAND
20638,-121.32,39.43,18.0,1860.0,409.0,741.0,349.0,1.8672,84700.0,INLAND


In [7]:
df_House.head()
# Fill missing values with zeros.
df_House=df_House.fillna(0)
# Apply the log transform to median_house_value.
df_House['median_house_value']=np.log1p(df_House['median_house_value'])


# Do train/validation/test split with 60%/20%/20% distribution.
# @ SPLITTING THE DATASET:
from sklearn.model_selection import train_test_split
df_full_train, df_test = train_test_split(df_House, test_size=0.2, random_state=1)
df_train, df_val = train_test_split(df_full_train, test_size=0.25, random_state=1)
y_train=df_train['median_house_value'].values
y_val=df_val['median_house_value'].values
y_test=df_test['median_house_value'].values
del df_train['median_house_value']

# Use the train_test_split function and set the random_state parameter to 1.
# Use DictVectorizer(sparse=True) to turn the dataframes into matrices.
print(df_train.shape, df_val.shape, df_test.shape)



(9411, 9) (3138, 10) (3138, 10)


In [8]:
print(y_train.shape, y_val.shape, y_test.shape)

(9411,) (3138,) (3138,)


In [10]:
from sklearn.feature_extraction import DictVectorizer
train_dicts = df_train.to_dict(orient='records')
dv = DictVectorizer(sparse=True)
X_train = dv.fit_transform(train_dicts)
val_dicts = df_val.to_dict(orient='records')
X_val = dv.transform(val_dicts)
test_dicts = df_test.to_dict(orient='records')
X_test = dv.transform(test_dicts)
print(X_train.shape, X_val.shape, X_test.shape)


(9411, 10) (3138, 10) (3138, 10)


Question 1<br>
Let's train a decision tree regressor to predict the median_house_value variable.<br>
<br>
Train a model with max_depth=1.<br>
Which feature is used for splitting the data?<br>
<br>
ocean_proximity<br>
total_rooms<br>
latitude<br>
population<br>

In [19]:
from sklearn.tree import DecisionTreeRegressor
from sklearn.metrics import roc_auc_score
from sklearn.metrics import mean_squared_error
from sklearn.tree import export_text
dt=DecisionTreeRegressor(max_depth=1)
dt.fit(X_train,y_train)

DecisionTreeRegressor(max_depth=1)

In [32]:
# Get the feature used for splitting the data
feature_idx = dt.tree_.feature[0]
feature_idx
feature_name = df_train.columns[feature_idx-1]
print('Feature used for splitting the data:', feature_name)

Feature used for splitting the data: population


In [29]:
df_train.columns


Index(['longitude', 'latitude', 'housing_median_age', 'total_rooms',
       'total_bedrooms', 'population', 'households', 'median_income',
       'ocean_proximity'],
      dtype='object')

Question 2

In [33]:
from sklearn.ensemble import RandomForestRegressor

# Train the model
rf = RandomForestRegressor(n_estimators=10, random_state=1, n_jobs=-1)
rf.fit(X_train, y_train)

RandomForestRegressor(n_estimators=10, n_jobs=-1, random_state=1)

In [34]:
# Make predictions on the validation data
y_pred = rf.predict(X_val)

# Calculate the RMSE
rmse = np.sqrt(mean_squared_error(y_val, y_pred))
print('RMSE', rmse)

RMSE 0.018834162984234817


In [35]:
from sklearn.metrics import mean_squared_error
import numpy as np

# Define a list of n_estimators values to try
n_estimators_values = list(range(10, 201, 10))

# Initialize a list to store RMSE values for each n_estimators
rmse_values = []

# Loop over n_estimators_values
for n in n_estimators_values:
    # Train the model with the current n_estimators
    rf = RandomForestRegressor(n_estimators=n, random_state=1, n_jobs=-1)
    rf.fit(X_train, y_train)

    # Make predictions on the validation data
    y_pred = rf.predict(X_val)

    # Calculate the RMSE and append to rmse_values
    rmse = np.sqrt(mean_squared_error(y_val, y_pred))
    rmse_values.append(rmse)

KeyboardInterrupt: 

In [38]:
from sklearn.ensemble import RandomForestRegressor

# Train the model with the specified parameters
rf = RandomForestRegressor(n_estimators=10, max_depth=20, random_state=1, n_jobs=-1)
rf.fit(X_train, y_train)

# Get the feature importance information
importances = rf.feature_importances_

# Get the most important feature
most_important_feature_index = np.argmax(importances)
most_important_feature = df_train.columns[most_important_feature_index]

print('Most important feature:', most_important_feature)

Most important feature: total_bedrooms
